In [1]:
!pip install --no-deps /kaggle/input/open-clip-dependencies-2/timm-1.0.22-py3-none-any.whl
!pip install --no-deps /kaggle/input/protobuf528-1209/protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl

Processing /kaggle/input/open-clip-dependencies-2/timm-1.0.22-py3-none-any.whl
  Attempting uninstall: timm
    Found existing installation: timm 1.0.19
    Uninstalling timm-1.0.19:
      Successfully uninstalled timm-1.0.19
Processing /kaggle/input/protobuf528-1209/protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


In [2]:
%%writefile infer_single.py

# 📦
import os
import gc
import cv2
import json
import time
import subprocess
from pathlib import Path
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random

from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.amp import autocast, GradScaler       

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import timm


dir = Path('/kaggle/input/csiro-biomass')
DIRS = {
    "dir"  : dir,
    "train": Path(dir, "train"),
    "test" : Path(dir, "test"),
    "model": Path('/kaggle/input/single-1209-swa-models'),
    "data" : Path("/kaggle/working/"),
}


def show_df_info(df, name: str):
    print(f"📊 {name:<16} shape: {str(df.shape):<16}  列名: {df.columns.tolist()}")


def move_column_first(df, col_name):
    if col_name not in df.columns:
        raise ValueError(f"列 '{col_name}' 不存在于 DataFrame 中。")

    cols = [col_name] + [c for c in df.columns if c != col_name]
    return df[cols]


def setup_seed(seed, deterministic=True):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = deterministic
    torch.backends.cudnn.benchmark = False
    

# Model 
class MySingleStreamModel(nn.Module):
    def __init__(self, 
                backbone_name="convnext_tiny", 
                pretrained=True, 
                config = None):
        super().__init__()
        print("Current backbone:", backbone_name)
        img_size = (config["img_size"], config.get("img_width", config["img_size"]))
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=pretrained,
            num_classes=0,
            img_size=img_size
        )
        in_dim = self.backbone.num_features

        params = list(self.backbone.parameters())
        freeze_until = int(len(params) * config["freeze_ratio"])
        for i, p in enumerate(params):
            p.requires_grad = i >= freeze_until 

        self.feature_dim = in_dim
        print("feature dim = ", self.feature_dim)
        def make_head():
            return nn.Sequential(
                nn.Linear(self.feature_dim, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Linear(128, 1)
            )
        self.head_green  = make_head()
        self.head_clover = make_head()
        self.head_dead   = make_head()
        self.head_gdm = make_head()
        self.head_total = make_head()
        self.softplus = nn.Softplus(beta=1.0)

    def forward(self, img):
        feat = self.backbone(img)
        
        G = self.softplus(self.head_green(feat))
        C = self.softplus(self.head_clover(feat))
        D = self.softplus(self.head_dead(feat))
        # G = torch.where(G < 0.4, torch.zeros_like(G), G)
        # C = torch.where(C < 0.4, torch.zeros_like(C), C)
        # D = torch.where(D < 0.4, torch.zeros_like(D), D)
        GDM   = self.softplus(self.head_gdm(feat))
        Total = self.softplus(self.head_total(feat))

        preds = torch.cat([G, C, D, GDM, Total], dim=1)
        return preds

    def compute_loss(self, preds, targets):
        preds = preds.view(-1, 5)
        targets = targets.view(-1, 5)
        weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5], device=preds.device)
        y_true_flat = targets.view(-1)
        y_pred_flat = preds.view(-1)
        w_flat = torch.cat([
            torch.full_like(targets[:, i], weights[i], device=preds.device)
            for i in range(5)
        ])
        y_mean = torch.sum(w_flat * y_true_flat) / torch.sum(w_flat)
        ss_res = torch.sum(w_flat * (y_true_flat - y_pred_flat) ** 2)
        ss_tot = torch.sum(w_flat * (y_true_flat - y_mean) ** 2)
        loss = ss_res / ss_tot
        return loss


class SingleStreamDataset(Dataset):
    def __init__(self, df, image_dir, config, pre_transform=None, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.target_cols = config["target_cols"]
        self.pre_transform = pre_transform
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = Path(self.image_dir, str(row["image_path"]))
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"⚠️ Failed to read image: {img_path} ({e})")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)

        image = np.array(image)
        if self.pre_transform:
            image = self.pre_transform(image=image)["image"]
        
        if self.transform:
            image = self.transform(image=image)["image"]

        if self.target_cols is not None:
            targets = torch.tensor(
                row[self.target_cols].astype(float).values,
                dtype=torch.float32
            )
            return image, targets
        else:
            return image


def get_pre_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(
                brightness_limit=(0.8, 1.2), 
                contrast_limit=(0.8, 1.2),   
                p=0.3
            ),
            A.ColorJitter(p=0.7)
        ], p=0.4),
    ])


def _get_resize_dims(config):
    height = config["img_size"]
    width = int(config.get("img_width", config["img_size"]))
    return height, width


def get_train_transforms(config):
    height, width = _get_resize_dims(config)
    return A.Compose([
        A.Resize(height, width),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_valid_transforms(config):
    height, width = _get_resize_dims(config)
    return A.Compose([
        A.Resize(height, width),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_tta_transforms(config):
    height, width = _get_resize_dims(config)
    return {
        "base": A.Compose([
            A.Resize(height, width),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        "hflip": A.Compose([
            A.HorizontalFlip(p=1.0)
        ])
    }


config = {
    # ⚙️ basic train params
    "seed"                           : 43,  
    "freeze_ratio"                   : 0.8,
    "num_workers"                    : 4,
    "prefetch_factor"                : 3,
    "batch_size"                     : 4,
    "img_size"                       : 1024,
    "img_ratio"                      : 2.0,
    "backbone_name"                  : "vit_large_patch16_dinov3_qkvb.lvd1689m",

    "target_cols": [
        "Dry_Green_g",
        "Dry_Clover_g",
        "Dry_Dead_g",
        "GDM_g",
        "Dry_Total_g"
    ]
}
config["img_width"] = int(config["img_size"] * config.get("img_ratio", 1.0))
isPREDICT   = True
DEBUG       = False


def load_and_prepare_test_df():
    df_file_path = Path(DIRS["dir"]) / "test.csv"
    df = pd.read_csv(df_file_path)
    df["ID"] = df["sample_id"].str.split("__").str[0]
    df = move_column_first(df, "ID")
    df["target"] = 0  
    df_targets = (
        df.pivot_table(index="ID", columns="target_name", values="target", aggfunc="first")
        .reset_index()
    )
    df_targets.columns.name = None
    df_meta = df[["ID", "image_path"]].drop_duplicates(subset="ID")
    df_test = pd.merge(df_meta, df_targets, on="ID", how="left")
    show_df_info(df_test, "df_test")
    print(f"✅ Test set loaded: {df_test.shape}")
    return df_test


def load_model_for_inference(model_path, device, config):
    model = MySingleStreamModel(config["backbone_name"], pretrained=False, config=config)
    model = model.to(device).to(memory_format=torch.channels_last)
    try:
        state_dict = torch.load(model_path, map_location=device, weights_only=True)
    except TypeError:
        state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model


def predict_with_model(model, dataloader, device):
    preds_list = []
    model.eval()
    with torch.no_grad():
        for step, (images, targets) in enumerate(dataloader):
            images = images.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last)
            targets = targets.to(device, non_blocking=True)
            with torch.amp.autocast(device_type='cuda'):
                preds = model(images)
            preds_list.append(preds.cpu().numpy())
    return np.concatenate(preds_list, axis=0)


def predict_ensemble(df_test, pre_transform, transform, model_dir, device, config):
    model_paths = sorted(Path(model_dir).glob("*.pt"))
    assert len(model_paths) > 0, f"❌ No model files found in: {model_dir}"

    print(f"✅ Detected {len(model_paths)} 个模型:")
    for p in model_paths:
        print("   -", p.name)
    test_dataset = SingleStreamDataset(
        df_test,
        DIRS["dir"],
        config,
        pre_transform=pre_transform,
        transform=transform,
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )
    fold_preds = []
    for fold, model_path in enumerate(model_paths):
        print(f"Fold {fold+1}/{len(model_paths)} 推理: {model_path.name}")
        model = load_model_for_inference(model_path, device, config)
        fold_pred = predict_with_model(model, test_loader, device)
        fold_preds.append(fold_pred)
    preds_mean = np.mean(fold_preds, axis=0)
    df_pred = pd.DataFrame(preds_mean, columns=config["target_cols"])
    df_pred["ID"] = df_test["ID"]
    df_pred = df_pred[["ID"] + config["target_cols"]]
    show_df_info(df_pred, "df_pred")
    return df_pred


def generate_submission(df_pred_final):
    ordered_target_cols = [
        "Dry_Green_g",  
        "Dry_Clover_g", 
        "Dry_Dead_g",  
        "GDM_g",       
        "Dry_Total_g"  
    ]

    if "Dry_Clover_g" in df_pred_final.columns:
        df_pred_final["Dry_Clover_g"] = df_pred_final["Dry_Clover_g"] * 0.8
    else:
        print("   ⚠️ Warning: Column Dry_Clover_g not found in DataFrame")
        

    df_submit = (
        df_pred_final.melt(
            id_vars="ID",
            value_vars=ordered_target_cols,
            var_name="target_name",
            value_name="target"
        )
    )

    df_submit["sample_id"] = df_submit["ID"] + "__" + df_submit["target_name"]
    df_submit = df_submit[["sample_id", "target"]]
    df_submit = df_submit.sort_values("sample_id").reset_index(drop=True)
    print(df_submit.head(10))
    df_submit.to_csv("submission1.csv", index=False)
    print("\n✅ Submission file generated: submission1.csv")


def run_tta_prediction(df_test, model_dir, device, config):
    tta_transforms = get_tta_transforms(config)
    tta_names = list(tta_transforms.keys())
    print(f"\n✅ Detected {len(tta_names)} TTA modes: {tta_names}\n")

    all_preds = []
    for name, _ in tta_transforms.items():
        print(f"\n🚀 TTA mode: {name}")
        if name == "base":
            pre_transform = None
            transform = tta_transforms[name]
        else:
            pre_transform = tta_transforms[name]
            transform = tta_transforms["base"]
        df_pred = predict_ensemble(df_test, pre_transform, transform, model_dir, device, config)
        all_preds.append(df_pred[config["target_cols"]].values)

    mean_preds = np.mean(all_preds, axis=0)
    df_pred_final = df_pred.copy()
    df_pred_final[config["target_cols"]] = mean_preds
    return df_pred_final


if __name__ == "__main__" and isPREDICT:
    print("\n🧠 Starting prediction pipeline...")
    setup_seed(config["seed"])    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    df_test = load_and_prepare_test_df()
    print(f"\nCurrent dataset shape: {df_test.shape}\n")

    model_dir = DIRS["model"]
    print(f"Model directory loaded: {model_dir}")

    df_pred_final = run_tta_prediction(df_test, model_dir, device, config)
    generate_submission(df_pred_final)

Writing infer_single.py


In [3]:
%%writefile infer_dual.py

# 📦
import os
import gc
import cv2
import json
import time
import subprocess
from pathlib import Path
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random

from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.amp import autocast, GradScaler       

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import timm

dir = Path('/kaggle/input/csiro-biomass')
DIRS = {
    "dir"  : dir,
    "train": Path(dir, "train"),
    "test" : Path(dir, "test"),
    "model": Path('/kaggle/input', "dinvov3large-1204-cv"),
    "data" : Path("/kaggle/working/"),
}


def setup_seed(seed, deterministic=True):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = deterministic
    torch.backends.cudnn.benchmark = False
    

def show_df_info(df, name: str):
    print(f"📊 {name:<16} shape: {str(df.shape):<16}  列名: {df.columns.tolist()}")


def move_column_first(df, col_name):
    if col_name not in df.columns:
        raise ValueError(f"列 '{col_name}' 不存在于 DataFrame 中。")
    cols = [col_name] + [c for c in df.columns if c != col_name]
    return df[cols]

    
class MyDualStreamModel(nn.Module):
    def __init__(self, 
                backbone_name="convnext_tiny", 
                pretrained=True, 
                config = None):
        super().__init__()
        print("Current backbone:", backbone_name)
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0)
        in_dim = self.backbone.num_features

        params = list(self.backbone.parameters())
        freeze_until = int(len(params) * config["freeze_ratio"])
        for i, p in enumerate(params):
            p.requires_grad = i >= freeze_until     
        self.fusion_dim = in_dim * 2
        print("feature fusion dim = ", self.fusion_dim)
        def make_head():
            return nn.Sequential(
                nn.Linear(self.fusion_dim, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Linear(128, 1)
            )
        self.head_green  = make_head()
        self.head_clover = make_head()
        self.head_dead   = make_head()
        self.head_gdm = make_head()
        self.head_total = make_head()
        self.softplus = nn.Softplus(beta=1.0)

    def forward(self, img_left, img_right):
        feat_left  = self.backbone(img_left)
        feat_right = self.backbone(img_right)
        fused = torch.cat([feat_left, feat_right], dim=1)
        G = self.softplus(self.head_green(fused))
        C = self.softplus(self.head_clover(fused))
        D = self.softplus(self.head_dead(fused))
        # G = torch.where(G < 0.4, torch.zeros_like(G), G)
        # C = torch.where(C < 0.4, torch.zeros_like(C), C)
        # D = torch.where(D < 0.4, torch.zeros_like(D), D)
        GDM   = G + C
        Total = G + C + D
        preds = torch.cat([G, C, D, GDM, Total], dim=1)
        return preds


class DualStreamDataset(Dataset):
    def __init__(self, df, image_dir, config, pre_transform=None, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.target_cols = config["target_cols"]
        self.pre_transform = pre_transform
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = Path(self.image_dir, str(row["image_path"]))
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"⚠️ Failed to read image: {img_path} ({e})")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)

        image = np.array(image)
        if self.pre_transform:
            image = self.pre_transform(image=image)["image"]
        h, w, _ = image.shape
        mid = w // 2
        img_left = image[:, :mid]
        img_right = image[:, mid:]
        if self.transform:
            img_left = self.transform(image=img_left)["image"]
            img_right = self.transform(image=img_right)["image"]
        if self.target_cols is not None:
            targets = torch.tensor(
                row[self.target_cols].astype(float).values,
                dtype=torch.float32
            )
            return img_left, img_right, targets
        else:
            return img_left, img_right


def get_pre_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(
                brightness_limit=(0.8, 1.2), 
                contrast_limit=(0.8, 1.2),   
                p=0.3
            ),
            A.ColorJitter(p=0.7)
        ], p=0.4),
    ])


def get_train_transforms(size):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_valid_transforms(size):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_tta_transforms(size):
    return {
        "base": A.Compose([
            A.Resize(size, size),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        "hflip": A.Compose([
            A.HorizontalFlip(p=1.0)
        ])
    }


config = {
    "seed"                           : 43,  
    "freeze_ratio"                   : 0.8,
    "num_workers"                    : 4,
    "prefetch_factor"                : 3,
    "batch_size"                     : 8,
    "img_size"                       : 1024,
    "backbone_name"                  : "vit_large_patch16_dinov3_qkvb.lvd1689m",
    "target_cols": [
        "Dry_Green_g",
        "Dry_Clover_g",
        "Dry_Dead_g",
        "GDM_g",
        "Dry_Total_g"
    ]
}
isPREDICT   = True
DEBUG       = False


def load_and_prepare_test_df():
    df_file_path = Path(DIRS["dir"]) / "test.csv"
    df = pd.read_csv(df_file_path)

    df["ID"] = df["sample_id"].str.split("__").str[0]
    df = move_column_first(df, "ID")
    df["target"] = 0  

    df_targets = (
        df.pivot_table(index="ID", columns="target_name", values="target", aggfunc="first")
        .reset_index()
    )
    df_targets.columns.name = None

    df_meta = df[["ID", "image_path"]].drop_duplicates(subset="ID")
    df_test = pd.merge(df_meta, df_targets, on="ID", how="left")
    show_df_info(df_test, "df_test")

    print(f"✅ Test set loaded: {df_test.shape}")
    return df_test


def load_model_for_inference(model_path, device, config):
    model = MyDualStreamModel(config["backbone_name"], pretrained=False, config=config)
    model = model.to(device).to(memory_format=torch.channels_last)
    try:
        state_dict = torch.load(model_path, map_location=device, weights_only=True)
    except TypeError:
        state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model


def predict_with_model(model, dataloader, device):
    preds_list = []
    model.eval()
    with torch.no_grad():
        for step, (img_left, img_right, targets) in enumerate(dataloader):
            img_left   = img_left.to(device, non_blocking=True)
            img_right  = img_right.to(device, non_blocking=True)
            targets    = targets.to(device, non_blocking=True)
            with torch.amp.autocast(device_type='cuda'):
                preds = model(img_left, img_right)
            preds_list.append(preds.cpu().numpy())
    return np.concatenate(preds_list, axis=0)


def predict_ensemble(df_test, pre_transform, transform, model_dir, device, config):
    model_paths = sorted(Path(model_dir).glob("fold*.pt"))
    assert len(model_paths) > 0, f"❌ No model files found in: {model_dir}"
    print(f"✅ Detected {len(model_paths)} 个模型:")
    for p in model_paths:
        print("   -", p.name)
    test_dataset = DualStreamDataset(
        df_test,
        DIRS["dir"],
        config,
        pre_transform=pre_transform,
        transform=transform,
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )
    fold_preds = []
    for fold, model_path in enumerate(model_paths):
        print(f"Fold {fold+1}/{len(model_paths)} 推理: {model_path.name}")
        model = load_model_for_inference(model_path, device, config)
        fold_pred = predict_with_model(model, test_loader, device)
        fold_preds.append(fold_pred)
    preds_mean = np.mean(fold_preds, axis=0)
    df_pred = pd.DataFrame(preds_mean, columns=config["target_cols"])
    df_pred["ID"] = df_test["ID"]
    df_pred = df_pred[["ID"] + config["target_cols"]]
    show_df_info(df_pred, "df_pred")
    return df_pred


def generate_submission(df_pred_final):
    ordered_target_cols = [
        "Dry_Green_g",  
        "Dry_Clover_g", 
        "Dry_Dead_g",   
        "GDM_g",        
        "Dry_Total_g"   
    ]
    if "Dry_Clover_g" in df_pred_final.columns:
        df_pred_final["Dry_Clover_g"] = df_pred_final["Dry_Clover_g"] * 0.8
    else:
        print("   ⚠️ Warning: Column Dry_Clover_g not found in DataFrame")
        
    df_submit = (
        df_pred_final.melt(
            id_vars="ID",
            value_vars=ordered_target_cols,
            var_name="target_name",
            value_name="target"
        )
    )
    df_submit["sample_id"] = df_submit["ID"] + "__" + df_submit["target_name"]
    df_submit = df_submit[["sample_id", "target"]]
    df_submit = df_submit.sort_values("sample_id").reset_index(drop=True)
    print(df_submit.head(10))
    df_submit.to_csv("submission2.csv", index=False)
    print("\n✅ Submission file generated: submission2.csv")


def run_tta_prediction(df_test, model_dir, device, config):
    tta_transforms = get_tta_transforms(config["img_size"])
    tta_names = list(tta_transforms.keys())
    print(f"\n✅ Detected {len(tta_names)} TTA modes: {tta_names}\n")
    all_preds = []
    for name, _ in tta_transforms.items():
        print(f"\n🚀 TTA mode: {name}")
        if name == "base":
            pre_transform = None
            transform = tta_transforms[name]
        else:
            pre_transform = tta_transforms[name]
            transform = tta_transforms["base"]
        df_pred = predict_ensemble(df_test, pre_transform, transform, model_dir, device, config)
        all_preds.append(df_pred[config["target_cols"]].values)
    mean_preds = np.mean(all_preds, axis=0)
    df_pred_final = df_pred.copy()
    df_pred_final[config["target_cols"]] = mean_preds
    return df_pred_final


if __name__ == "__main__" and isPREDICT:
    print("\n🧠 Starting prediction pipeline...")
    setup_seed(config["seed"])    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   
    df_test = load_and_prepare_test_df()
    print(f"\nCurrent dataset shape: {df_test.shape}\n")
    
    model_dir = DIRS["model"]
    print(f"Model directory loaded: {model_dir}")

    df_pred_final = run_tta_prediction(df_test, model_dir, device, config)
    generate_submission(df_pred_final)

Writing infer_dual.py


In [4]:
%%bash
CUDA_VISIBLE_DEVICES=0 python infer_single.py > log_single.txt 2>&1 &
CUDA_VISIBLE_DEVICES=1 python infer_dual.py > log_dual.txt 2>&1 &
echo "GPU 0: Single Stream"
echo "GPU 1: Dual Stream"

wait

echo "single&&dual done！"

GPU 0: Single Stream
GPU 1: Dual Stream
single&&dual done！


In [5]:
!tail -n 1000 log_single.txt

/usr/local/lib/python3.11/dist-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context

In [6]:
!tail -n 1000 log_dual.txt

/usr/local/lib/python3.11/dist-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context

In [7]:
import gc
import torch

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [8]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('submission1.csv')
df2 = pd.read_csv('submission2.csv')

df1 = df1.sort_values('sample_id').reset_index(drop=True)
df2 = df2.sort_values('sample_id').reset_index(drop=True)

if not df1['sample_id'].equals(df2['sample_id']):
    print("警告：两个文件的sample_id不完全一致！")
    print("将按sample_id对齐...")
    df_merged = pd.merge(df1, df2, on='sample_id', suffixes=('_1', '_2'))
else:
    df_merged = df1.copy()
    df_merged['target_1'] = df1['target']
    df_merged['target_2'] = df2['target']


df_merged['target'] = 0.8 * df_merged['target_1'] + 0.2 * df_merged['target_2']
df_merged['target'] = df_merged['target'].apply(lambda x: max(x, 0) if x >= 0.1 else 0)
df_merged['target'] = df_merged['target'].clip(lower=0)
result_df = df_merged[['sample_id', 'target']].copy()

negative_count = (result_df['target'] < 0).sum()
below_01_count = ((df_merged['target_1'] < 0.1) | (df_merged['target_2'] < 0.1)).sum()
clipped_count = (result_df['target'] == 0).sum()

print(f"处理后统计:")
print(f"  负值数量: {negative_count}")
print(f"  原始值低于0.1的数量: {below_01_count}")
print(f"  被置为0的数量: {clipped_count}")
print(f"  最终值范围: [{result_df['target'].min():.3f}, {result_df['target'].max():.3f}]")

result_df.to_csv('submission_stage1.csv', index=False)
print(f"\n✅ 提交文件已保存: submission.csv")

print(result_df.head(10))

处理后统计:
  负值数量: 0
  原始值低于0.1的数量: 1
  被置为0的数量: 0
  最终值范围: [0.126, 65.567]

✅ 提交文件已保存: submission.csv
                    sample_id     target
0  ID1001187975__Dry_Clover_g   0.125842
1    ID1001187975__Dry_Dead_g  29.677812
2   ID1001187975__Dry_Green_g  35.810001
3   ID1001187975__Dry_Total_g  65.567343
4         ID1001187975__GDM_g  35.707031


# stage2 online training

In [9]:
%%writefile train_single.py

# 📦
import os
import gc
import cv2
import json
import time
import shutil
import subprocess
from pathlib import Path
from datetime import datetime, timedelta
import math
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import r2_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import _LRScheduler
from torch.amp import autocast, GradScaler       
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import timm
from torchvision import models, transforms
from torchvision.models import get_model_weights
import collections


dir = Path('/kaggle/input/csiro-biomass')
DIRS = {
    "dir"  : dir,
    "train": Path(dir, "train"),
    "test" : Path(dir, "test"),
    "model": Path('/kaggle/working/stage2_train_single_swa'),
    "save" : Path('/kaggle/working/stage2_train_single'),
    "data" : Path("/kaggle/working/"),
}


def show_df_info(df, name: str):
    print(f"📊 {name:<16} shape: {str(df.shape):<16}  列名: {df.columns.tolist()}")


def move_column_first(df, col_name):
    if col_name not in df.columns:
        raise ValueError(f"列 '{col_name}' 不存在于 DataFrame 中。")
    cols = [col_name] + [c for c in df.columns if c != col_name]
    return df[cols]


def setup_seed(seed, deterministic=True):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = deterministic
    torch.backends.cudnn.benchmark = False


def load_and_prepare_train_stage2_df():
    df_file_path = 'submission_stage1.csv'
    df = pd.read_csv(df_file_path)
    df["ID"] = df["sample_id"].str.split("__").str[0]
    df["image_path"] = df["ID"].apply(lambda x: f"test/{x}.jpg")
    df["target_name"] = df["sample_id"].str.split("__").str[1]
    df = move_column_first(df, "ID")

    df_targets = (
        df.pivot_table(
            index="ID",
            columns="target_name",
            values="target",
            aggfunc="first"
        )
        .reset_index()
    )
    df_targets.columns.name = None  
    meta_cols = [
        "ID", "image_path"
    ]
    df_meta = df[meta_cols].drop_duplicates(subset="ID")
    df_train = pd.merge(df_meta, df_targets, on="ID", how="left")
    show_df_info(df_train, "df_train")
    return df_train


df_pseudo = load_and_prepare_train_stage2_df()
df_stage1_path = '/kaggle/input/train-single-csv/train_with_folds_singleflow.csv'
df_stage1 = pd.read_csv(df_stage1_path)
df_stage2_train = pd.concat([df_pseudo, df_stage1], ignore_index=True)
df_stage2_train.reset_index(drop=True, inplace=True)


config = {
    "seed"                           : 42,  
    "epochs"                         : 12,
    "freeze_ratio"                   : 0.8,
    "num_workers"                    : 4,
    "prefetch_factor"                : 2,
    "batch_size"                     : 4,
    "lr"                             : 1e-4,
    "img_size"                       : 1024,
    "img_ratio"                      : 2.0, 
    "backbone_name"                  : "vit_large_patch16_dinov3_qkvb.lvd1689m",
    "scheduler_type"                 : "cosine",
    "weights": {
        "Dry_Green_g" : 0.1,
        "Dry_Clover_g": 0.1,
        "Dry_Dead_g"  : 0.1,
        "GDM_g"       : 0.2,
        "Dry_Total_g" : 0.5
    },
    "target_cols": [
        "Dry_Green_g",
        "Dry_Clover_g",
        "Dry_Dead_g",
        "GDM_g",
        "Dry_Total_g"
    ]
}
config["img_width"] = int(config["img_size"] * config.get("img_ratio", 1.0))


class MySingleStreamModel(nn.Module):
    def __init__(self, 
                backbone_name="convnext_tiny", 
                pretrained=True, 
                config = None):
        super().__init__()
        print("Current backbone:", backbone_name)
        img_size = (config["img_size"], config.get("img_width", config["img_size"]))
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=pretrained,
            num_classes=0,
            img_size=img_size
        )
        in_dim = self.backbone.num_features
        params = list(self.backbone.parameters())
        freeze_until = int(len(params) * config["freeze_ratio"])
        for i, p in enumerate(params):
            p.requires_grad = i >= freeze_until
        self.feature_dim = in_dim
        print("feature dim = ", self.feature_dim)
        def make_head():
            return nn.Sequential(
                nn.Linear(self.feature_dim, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Linear(128, 1)
            )
        self.head_green  = make_head()
        self.head_clover = make_head()
        self.head_dead   = make_head()
        self.head_gdm = make_head()
        self.head_total = make_head()
        self.softplus = nn.Softplus(beta=1.0)
        self.weights = config["weights"]

    def forward(self, img):
        feat = self.backbone(img)
        G = self.softplus(self.head_green(feat))
        C = self.softplus(self.head_clover(feat))
        D = self.softplus(self.head_dead(feat))
        GDM   = self.softplus(self.head_gdm(feat))
        Total = self.softplus(self.head_total(feat))
        preds = torch.cat([G, C, D, GDM, Total], dim=1)  # [B, 5]
        return preds

    def compute_loss(self, preds, targets):
        preds = preds.view(-1, 5)
        targets = targets.view(-1, 5)
        weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5], device=preds.device)
        y_true_flat = targets.view(-1)
        y_pred_flat = preds.view(-1)
        w_flat = torch.cat([
            torch.full_like(targets[:, i], weights[i], device=preds.device)
            for i in range(5)
        ])
        y_mean = torch.sum(w_flat * y_true_flat) / torch.sum(w_flat)
        ss_res = torch.sum(w_flat * (y_true_flat - y_pred_flat) ** 2)
        ss_tot = torch.sum(w_flat * (y_true_flat - y_mean) ** 2)
        loss = ss_res / ss_tot
        return loss


class SingleStreamDataset(Dataset):
    def __init__(self, df, image_dir, config, pre_transform=None, transform=None, mode="train"):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.target_cols = config["target_cols"]
        self.pre_transform = pre_transform
        self.transform = transform
        self.mode = mode

    def __len__(self):
        if self.mode == "train":
            return len(self.df)
        else:
            return len(self.df) 
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = Path(self.image_dir, str(row["image_path"]))
        image = Image.open(img_path).convert("RGB")
        image = np.array(image)
        if self.pre_transform:
            image = self.pre_transform(image=image)["image"]
        
        if self.transform:
            image = self.transform(image=image)["image"]

        if self.target_cols is not None:
            targets = torch.tensor(
                row[self.target_cols].astype(float).values,
                dtype=torch.float32
            )
            return image, targets
        else:
            return image


def get_pre_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(
                brightness_limit=(0.8, 1.2), 
                contrast_limit=(0.8, 1.2),   
                p=0.3
            ),
            A.ColorJitter(p=0.7)
        ], p=0.4),
    ])


def _get_resize_dims(config):
    height = config["img_size"]
    width = int(config.get("img_width", config["img_size"]))
    return height, width


def get_train_transforms(config):
    height, width = _get_resize_dims(config)
    return A.Compose([
        A.Resize(height, width),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_valid_transforms(config):
    height, width = _get_resize_dims(config)
    return A.Compose([
        A.Resize(height, width),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_tta_transforms(config):
    height, width = _get_resize_dims(config)
    return {
        "base": A.Compose([
            A.Resize(height, width),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        "hflip": A.Compose([
            A.HorizontalFlip(p=1.0)
        ])
    }


def train_one_epoch(model, dataloader, optimizer, device, scaler, config):
    model.train()
    running_loss = []
    start_epoch = time.time()
    prev_end = start_epoch  
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Training", leave=False)

    for step, (images, targets) in pbar:
        t_load = time.time()  
        data_load_time = t_load - prev_end
        t0 = time.time()
        images = images.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last)
        targets = targets.to(device, non_blocking=True)
        t1 = time.time()
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type='cuda'):
            preds = model(images)
            loss = model.compute_loss(preds, targets)    
        t2 = time.time()
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()
        t3 = time.time()
        running_loss.append(loss.item())
        prev_end = t3  
    end_epoch = time.time()
    epoch_time = end_epoch - start_epoch
    avg_batch_time = epoch_time / len(dataloader)
    return float(np.mean(running_loss))



def get_fold_loaders(df, config):
    train_df = df
    train_dataset = SingleStreamDataset(train_df, DIRS["dir"],
                                        config, pre_transform=get_pre_transforms(),
                                        transform=get_train_transforms(config), mode="train")
    num_workers = config["num_workers"]
    prefetch_factor = config["prefetch_factor"]
    train_loader = DataLoader(
        train_dataset,
        batch_size=config["batch_size"],
        shuffle=True,
        num_workers=num_workers,        
        pin_memory=True,                 
        prefetch_factor=prefetch_factor,  
        persistent_workers=True         
    )
    return train_loader


def fetch_scheduler(optimizer, config):
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config["epochs"], eta_min=config["lr"]/100)
    return scheduler


def train_one_fold(df, save_dir, config, device):   
    train_loader = get_fold_loaders(df, config)
    model = MySingleStreamModel(config["backbone_name"], pretrained=False, config=config)
    model = model.to(device).to(memory_format=torch.channels_last)
    try:
        state_dict = torch.load('/kaggle/input/single-1209-swa-models/fold2_swa.pt', map_location=device, weights_only=True)
    except TypeError:
        state_dict = torch.load('/kaggle/input/single-1209-swa-models/fold2_swa.pt', map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model = torch.compile(model)
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=config["lr"], weight_decay=2e-5)
    scheduler = fetch_scheduler(optimizer, config)
    scaler = torch.amp.GradScaler(device="cuda")
    train_losses, LR_records = [], []
    epoch_times = [] 
    all_progress = config["epochs"]
    for epoch in range(config["epochs"]):
        epoch_start = time.time()
        train_loss = train_one_epoch(model, train_loader, optimizer, device, scaler, config)
        scheduler.step()
        train_losses.append(train_loss)
        LR_records.append(scheduler.get_last_lr()[0])
        epoch_time = time.time() - epoch_start
        if epoch > 0:
            epoch_times.append(epoch_time)
            if len(epoch_times) > 10:
                epoch_times.pop(0)
        if len(epoch_times) > 0:
            avg_epoch_time = np.mean(epoch_times)
        else:
            avg_epoch_time = epoch_time

        progress = epoch + 1
        remaining_epochs = all_progress - progress
        eta_seconds = avg_epoch_time * remaining_epochs
        if not np.isnan(eta_seconds) and not np.isinf(eta_seconds):
            eta_time = datetime.now() + timedelta(seconds=float(eta_seconds))
            eta_time = eta_time.replace(microsecond=0)
            days_diff = (eta_time.date() - datetime.now().date()).days
            eta_str = (
                f"T+{days_diff} " + eta_time.strftime("%H:%M:%S")
                if days_diff > 0 else eta_time.strftime("%H:%M:%S")
            )
        else:
            eta_str = "--:--:--"
        now_str = datetime.now().strftime("%H:%M:%S")
        print(
            f"[{now_str}]🧩[{progress/all_progress*100:6.2f}%] "
            f"Epoch {epoch+1:03d}/{config['epochs']} | "
            f"Train={train_loss:.4f} | "
            f"LR={scheduler.get_last_lr()[0]:.6f} | "
            f"{epoch_time:6.2f}s/it | "
            f"ETA≈{eta_str}"
        )
        if epoch > 7:
            loss_model_path = save_dir / f"model_epoch{epoch+1}.pt"
            torch.save(model._orig_mod.state_dict(), loss_model_path)
            # torch.save(model.state_dict(), loss_model_path)
            print(f"🌟 save Epoch {epoch+1} 模型！")
    del train_loader, model, optimizer, scheduler, scaler
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    return train_losses


def train_with_groupkfold(df_train, save_dir, config, device):
    df = df_train.copy()
    metrics = train_one_fold(df, save_dir, config, device)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("🎯 All folds training completed.")


isTRAIN     = True
isPREDICT   = True
DEBUG = False
print("DEBUG模式",DEBUG)


if (
    __name__ == "__main__"
    and isTRAIN
):
    torch.multiprocessing.freeze_support()       
    setup_seed(config["seed"])        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_root = DIRS["save"]
    os.makedirs(model_root, exist_ok=True)
    if DEBUG:
        config["epochs"] = 2
        print(f"⚠️ DEBUG mode enabled — running only {config['epochs']} epochs\n")


    print("\n🚀🚀🚀 Starting training process... 🚀🚀🚀\n")
    df_train = df_stage2_train
    train_with_groupkfold(
        df_train             = df_train,
        save_dir             = model_root,
        config               = config,
        device               = device
    )
    print(f"\n✅ Training completed! Results saved in: {model_root}")
    print("✅"*65)

    model_dir = DIRS["save"]
    out_dir = DIRS["model"]
    model_paths = sorted(Path(model_dir).glob("*.pt"))
    print(f": Found {len(model_paths)} unique models!")
    for path in model_paths:
        print(f" - {path}")
    if len(model_paths) < 2:
        print(f"⚠️ Only {len(model_paths)} unique models, skipping SWA...")
    
    models = []
    for module_path in model_paths:
        if os.path.exists(module_path):
            model = torch.load(module_path, map_location='cpu')
            models.append(model)
        else:
            print(f"❌ Model file not found: {module_path}")
    
    worker_state_dicts = [m for m in models]
    weight_keys = list(worker_state_dicts[0].keys())
    print(f"Example weight keys: {list(weight_keys)[:5]}")
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        key_sum = 0
        for i in range(len(models)):
            key_sum += worker_state_dicts[i][key]
        fed_state_dict[key] = key_sum / len(models)
    os.makedirs(out_dir, exist_ok=True)
    output_path = os.path.join(out_dir, f'swa.pt')
    torch.save(fed_state_dict, output_path)
    print(f"averaging complete. Saved to: {output_path}")



def load_and_prepare_test_df():
    df_file_path = Path(DIRS["dir"]) / "test.csv"
    df = pd.read_csv(df_file_path)

    df["ID"] = df["sample_id"].str.split("__").str[0]
    df = move_column_first(df, "ID")
    df["target"] = 0  
    df_targets = (
        df.pivot_table(index="ID", columns="target_name", values="target", aggfunc="first")
        .reset_index()
    )
    df_targets.columns.name = None
    df_meta = df[["ID", "image_path"]].drop_duplicates(subset="ID")
    df_test = pd.merge(df_meta, df_targets, on="ID", how="left")
    show_df_info(df_test, "df_test")
    print(f"✅ Test set loaded: {df_test.shape}")
    return df_test


def load_model_for_inference(model_path, device, config):
    model = MySingleStreamModel(config["backbone_name"], pretrained=False, config=config)
    model = model.to(device).to(memory_format=torch.channels_last)
    try:
        state_dict = torch.load(model_path, map_location=device, weights_only=True)
    except TypeError:
        state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model


def predict_with_model(model, dataloader, device):
    preds_list = []
    model.eval()
    with torch.no_grad():
        for step, (images, targets) in enumerate(dataloader):
            images = images.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last)
            targets = targets.to(device, non_blocking=True)
            with torch.amp.autocast(device_type='cuda'):
                preds = model(images)
            preds_list.append(preds.cpu().numpy())
    return np.concatenate(preds_list, axis=0)


def predict_ensemble(df_test, pre_transform, transform, model_dir, device, config):
    model_paths = sorted(Path(model_dir).glob("*.pt"))
    assert len(model_paths) > 0, f"❌ No model files found in: {model_dir}"
    print(f"✅ Detected {len(model_paths)} 个模型:")
    for p in model_paths:
        print("   -", p.name)
    test_dataset = SingleStreamDataset(
        df_test,
        DIRS["dir"],
        config,
        pre_transform=pre_transform,
        transform=transform,
        mode="test"
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )
    fold_preds = []
    for fold, model_path in enumerate(model_paths):
        print(f"Fold {fold+1}/{len(model_paths)} 推理: {model_path.name}")
        model = load_model_for_inference(model_path, device, config)
        fold_pred = predict_with_model(model, test_loader, device)
        fold_preds.append(fold_pred)
    preds_mean = np.mean(fold_preds, axis=0)
    df_pred = pd.DataFrame(preds_mean, columns=config["target_cols"])
    df_pred["ID"] = df_test["ID"]
    df_pred = df_pred[["ID"] + config["target_cols"]]
    show_df_info(df_pred, "df_pred")
    return df_pred


def generate_submission(df_pred_final):
    ordered_target_cols = [
        "Dry_Green_g",  # 1️⃣
        "Dry_Clover_g", # 2️⃣
        "Dry_Dead_g",   # 3️⃣
        "GDM_g",        # 4️⃣
        "Dry_Total_g"   # 5️⃣
    ]
    # if "Dry_Clover_g" in df_pred_final.columns:
    #     df_pred_final["Dry_Clover_g"] = df_pred_final["Dry_Clover_g"] * 0.8
    # else:
    #     print("   ⚠️ Warning: Column Dry_Clover_g not found in DataFrame")

    df_submit = (
        df_pred_final.melt(
            id_vars="ID",
            value_vars=ordered_target_cols,
            var_name="target_name",
            value_name="target"
        )
    )

    df_submit["sample_id"] = df_submit["ID"] + "__" + df_submit["target_name"]
    df_submit = df_submit[["sample_id", "target"]]
    df_submit = df_submit.sort_values("sample_id").reset_index(drop=True)
    df_submit.to_csv("submission_stage2_1.csv", index=False)
    print("\n✅ Submission file generated: submission_stage2_1.csv")


def run_tta_prediction(df_test, model_dir, device, config):
    tta_transforms = get_tta_transforms(config)
    tta_names = list(tta_transforms.keys())
    print(f"\n✅ Detected {len(tta_names)} TTA modes: {tta_names}\n")
    all_preds = []
    for name, _ in tta_transforms.items():
        print(f"\n🚀 TTA mode: {name}")
        if name == "base":
            print("base no pre_transform")
            pre_transform = None
            transform = tta_transforms[name]
        else:
            pre_transform = tta_transforms[name]
            transform = tta_transforms["base"]
        df_pred = predict_ensemble(df_test, pre_transform, transform, model_dir, device, config)
        all_preds.append(df_pred[config["target_cols"]].values)

    mean_preds = np.mean(all_preds, axis=0)
    df_pred_final = df_pred.copy()
    df_pred_final[config["target_cols"]] = mean_preds
    return df_pred_final


if __name__ == "__main__" and isPREDICT:
    print("\n🧠 Starting prediction pipeline...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    df_test = load_and_prepare_test_df()
    print(f"\nCurrent dataset shape: {df_test.shape}\n")
    model_dir = DIRS["model"]

    print(f"Model directory loaded: {model_dir}")

    df_pred_final = run_tta_prediction(df_test, model_dir, device, config)
    generate_submission(df_pred_final)
    print("🎯 Prediction pipeline completed.")

Writing train_single.py


In [10]:
%%writefile train_dual.py

# 📦
import os
import gc
import cv2
import json
import time
import shutil
import subprocess
from pathlib import Path
from datetime import datetime, timedelta
import math
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import r2_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import _LRScheduler
from torch.amp import autocast, GradScaler       
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import timm
from torchvision import models, transforms
from torchvision.models import get_model_weights
import collections


dir = Path('/kaggle/input/csiro-biomass')
DIRS = {
    "dir"  : dir,
    "train": Path(dir, "train"),
    "test" : Path(dir, "test"),
    "model": Path('/kaggle/working/stage2_train_dual_swa'),
    "save" : Path('/kaggle/working/stage2_train_dual'),
    "data" : Path("/kaggle/working/"),
}


def show_df_info(df, name: str):
    print(f"📊 {name:<16} shape: {str(df.shape):<16}  列名: {df.columns.tolist()}")


def move_column_first(df, col_name):
    if col_name not in df.columns:
        raise ValueError(f"列 '{col_name}' 不存在于 DataFrame 中。")
    cols = [col_name] + [c for c in df.columns if c != col_name]
    return df[cols]


def setup_seed(seed, deterministic=True):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = deterministic
    torch.backends.cudnn.benchmark = False


def load_and_prepare_train_stage2_df():
    df_file_path = 'submission_stage1.csv'
    df = pd.read_csv(df_file_path)
    df["ID"] = df["sample_id"].str.split("__").str[0]
    df["image_path"] = df["ID"].apply(lambda x: f"test/{x}.jpg")
    df["target_name"] = df["sample_id"].str.split("__").str[1]
    df = move_column_first(df, "ID")

    df_targets = (
        df.pivot_table(
            index="ID",
            columns="target_name",
            values="target",
            aggfunc="first"
        )
        .reset_index()
    )
    df_targets.columns.name = None  
    meta_cols = [
        "ID", "image_path"
    ]
    df_meta = df[meta_cols].drop_duplicates(subset="ID")
    df_train = pd.merge(df_meta, df_targets, on="ID", how="left")
    show_df_info(df_train, "df_train")
    return df_train


df_pseudo = load_and_prepare_train_stage2_df()
df_stage1_path = '/kaggle/input/train-single-csv/train_with_folds_singleflow.csv'
df_stage1 = pd.read_csv(df_stage1_path)
df_stage2_train = pd.concat([df_pseudo, df_stage1], ignore_index=True)
df_stage2_train.reset_index(drop=True, inplace=True)


config = {
    "seed"                           : 3407,  
    "epochs"                         : 12,
    "freeze_ratio"                   : 0.8,
    "num_workers"                    : 2,
    "prefetch_factor"                : 2,
    "batch_size"                     : 4,
    "lr"                             : 1e-4,
    "img_size"                       : 1024,
    "backbone_name"                  : "vit_large_patch16_dinov3_qkvb.lvd1689m",
    "scheduler_type"                 : "cosine",
    "weights": {
        "Dry_Green_g" : 0.1,
        "Dry_Clover_g": 0.1,
        "Dry_Dead_g"  : 0.1,
        "GDM_g"       : 0.2,
        "Dry_Total_g" : 0.5
    },
    "target_cols": [
        "Dry_Green_g",
        "Dry_Clover_g",
        "Dry_Dead_g",
        "GDM_g",
        "Dry_Total_g"
    ]
}


# Model 
class MyDualStreamModel(nn.Module):
    def __init__(self, 
                backbone_name="convnext_tiny", 
                pretrained=True, 
                config = None):
        super().__init__()
        print("Current backbone:", backbone_name)
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0)
        in_dim = self.backbone.num_features
        params = list(self.backbone.parameters())
        freeze_until = int(len(params) * config["freeze_ratio"])
        for i, p in enumerate(params):
            p.requires_grad = i >= freeze_until    
        self.fusion_dim = in_dim * 2
        print("feature fusion dim = ", self.fusion_dim)
        def make_head():
            return nn.Sequential(
                nn.Linear(self.fusion_dim, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Linear(128, 1)
            )

        self.head_green  = make_head()  
        self.head_clover = make_head()   
        self.head_dead   = make_head()  
        self.head_gdm = make_head()
        self.head_total = make_head()
        self.softplus = nn.Softplus(beta=1.0)
        self.weights = config["weights"]
    def forward(self, img_left, img_right):
        feat_left  = self.backbone(img_left)
        feat_right = self.backbone(img_right)
        fused = torch.cat([feat_left, feat_right], dim=1)
        G = self.softplus(self.head_green(fused))
        C = self.softplus(self.head_clover(fused))
        D = self.softplus(self.head_dead(fused))
        if self.training:
            GDM   = self.softplus(self.head_gdm(fused))
            Total = self.softplus(self.head_total(fused))
        else:
            GDM   = G + C
            Total = G + C + D
        preds = torch.cat([G, C, D, GDM, Total], dim=1)  # [B, 5]
        return preds

    def compute_loss(self, preds, targets):
        preds = preds.view(-1, 5)
        targets = targets.view(-1, 5)
        weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5], device=preds.device)
        y_true_flat = targets.view(-1)
        y_pred_flat = preds.view(-1)
        w_flat = torch.cat([
            torch.full_like(targets[:, i], weights[i], device=preds.device)
            for i in range(5)
        ])
        y_mean = torch.sum(w_flat * y_true_flat) / torch.sum(w_flat)
        ss_res = torch.sum(w_flat * (y_true_flat - y_pred_flat) ** 2)
        ss_tot = torch.sum(w_flat * (y_true_flat - y_mean) ** 2)
        loss = ss_res / ss_tot
        return loss


class DualStreamDataset(Dataset):
    def __init__(self, df, image_dir, config, pre_transform=None, transform=None, mode="train"):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.target_cols = config["target_cols"]
        self.pre_transform = pre_transform
        self.transform = transform
        self.mode = mode

    def __len__(self):
        if self.mode == "train":
            return len(self.df)
        else:
            return len(self.df) 
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = Path(self.image_dir, str(row["image_path"]))
        image = Image.open(img_path).convert("RGB")
        image = np.array(image)
        if self.pre_transform:
            image = self.pre_transform(image=image)["image"]
        
        h, w, _ = image.shape
        mid = w // 2
        img_left = image[:, :mid]
        img_right = image[:, mid:]
        if self.transform:
            img_left = self.transform(image=img_left)["image"]
            img_right = self.transform(image=img_right)["image"]
        if self.target_cols is not None:
            targets = torch.tensor(
                row[self.target_cols].astype(float).values,
                dtype=torch.float32
            )
            return img_left, img_right, targets
        else:
            return img_left, img_right


def get_pre_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(
                brightness_limit=(0.8, 1.2), 
                contrast_limit=(0.8, 1.2),   
                p=0.3
            ),
            A.ColorJitter(p=0.7)
        ], p=0.4),
    ])


def get_train_transforms(size):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_valid_transforms(size):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_tta_transforms(size):
    return {
        "base": A.Compose([
            A.Resize(size, size),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        "hflip": A.Compose([
            A.HorizontalFlip(p=1.0)
        ])
    }


def train_one_epoch(model, dataloader, optimizer, device, scaler, config):
    model.train()
    running_loss = []
    start_epoch = time.time()
    prev_end = start_epoch  
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Training", leave=False)
    for step, (img_left, img_right, targets) in enumerate(dataloader):
        t_load = time.time()  
        data_load_time = t_load - prev_end
        t0 = time.time()
        img_left, img_right, targets = (
            img_left.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last),
            img_right.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last),
            targets.to(device, non_blocking=True),
        )
        t1 = time.time()
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type='cuda'):
            preds = model(img_left, img_right)
            loss = model.compute_loss(preds, targets)    
        t2 = time.time()
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()
        t3 = time.time()
        running_loss.append(loss.item())
        prev_end = t3  
    end_epoch = time.time()
    epoch_time = end_epoch - start_epoch
    avg_batch_time = epoch_time / len(dataloader)
    return float(np.mean(running_loss))



def get_fold_loaders(df, config):
    train_df = df
    train_dataset = DualStreamDataset(train_df, DIRS["dir"],
                                        config, pre_transform=get_pre_transforms(),
                                        transform=get_train_transforms(config["img_size"]), mode="train")
    num_workers = config["num_workers"]
    prefetch_factor = config["prefetch_factor"]
    train_loader = DataLoader(
        train_dataset,
        batch_size=config["batch_size"],
        shuffle=True,
        num_workers=num_workers,        
        pin_memory=True,                 
        prefetch_factor=prefetch_factor,  
        persistent_workers=True         
    )
    return train_loader


def fetch_scheduler(optimizer, config):
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config["epochs"], eta_min=config["lr"]/100)
    return scheduler


def train_one_fold(df, save_dir, config, device):   
    train_loader = get_fold_loaders(df, config)
    model = MyDualStreamModel(config["backbone_name"], pretrained=False, config=config)
    model = model.to(device).to(memory_format=torch.channels_last)
    try:
        state_dict = torch.load('/kaggle/input/dinvov3large-1204-cv/fold2_swa.pt', map_location=device, weights_only=True)
    except TypeError:
        state_dict = torch.load('/kaggle/input/dinvov3large-1204-cv/fold2_swa.pt', map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model = torch.compile(model)
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=config["lr"], weight_decay=2e-5)
    scheduler = fetch_scheduler(optimizer, config)
    scaler = torch.amp.GradScaler(device="cuda")
    train_losses, LR_records = [], []
    epoch_times = [] 
    all_progress = config["epochs"]
    for epoch in range(config["epochs"]):
        epoch_start = time.time()
        train_loss = train_one_epoch(model, train_loader, optimizer, device, scaler, config)
        scheduler.step()
        train_losses.append(train_loss)
        LR_records.append(scheduler.get_last_lr()[0])
        epoch_time = time.time() - epoch_start
        if epoch > 0:
            epoch_times.append(epoch_time)
            if len(epoch_times) > 10:
                epoch_times.pop(0)
        if len(epoch_times) > 0:
            avg_epoch_time = np.mean(epoch_times)
        else:
            avg_epoch_time = epoch_time

        progress = epoch + 1
        remaining_epochs = all_progress - progress
        eta_seconds = avg_epoch_time * remaining_epochs
        if not np.isnan(eta_seconds) and not np.isinf(eta_seconds):
            eta_time = datetime.now() + timedelta(seconds=float(eta_seconds))
            eta_time = eta_time.replace(microsecond=0)
            days_diff = (eta_time.date() - datetime.now().date()).days
            eta_str = (
                f"T+{days_diff} " + eta_time.strftime("%H:%M:%S")
                if days_diff > 0 else eta_time.strftime("%H:%M:%S")
            )
        else:
            eta_str = "--:--:--"
        now_str = datetime.now().strftime("%H:%M:%S")
        print(
            f"[{now_str}]🧩[{progress/all_progress*100:6.2f}%] "
            f"Epoch {epoch+1:03d}/{config['epochs']} | "
            f"Train={train_loss:.4f} | "
            f"LR={scheduler.get_last_lr()[0]:.6f} | "
            f"{epoch_time:6.2f}s/it | "
            f"ETA≈{eta_str}"
        )
        if epoch > 7:
            loss_model_path = save_dir / f"model_epoch{epoch+1}.pt"
            torch.save(model._orig_mod.state_dict(), loss_model_path)
            # torch.save(model.state_dict(), loss_model_path)
            print(f"🌟 save Epoch {epoch+1} 模型！")
    del train_loader, model, optimizer, scheduler, scaler
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    return train_losses


def train_with_groupkfold(df_train, save_dir, config, device):
    df = df_train.copy()
    metrics = train_one_fold(df, save_dir, config, device)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("🎯 All folds training completed.")


isTRAIN     = True
isPREDICT   = True
DEBUG = False
print("DEBUG模式",DEBUG)


if (
    __name__ == "__main__"
    and isTRAIN
):
    torch.multiprocessing.freeze_support()       
    setup_seed(config["seed"])        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_root = DIRS["save"]
    os.makedirs(model_root, exist_ok=True)
    if DEBUG:
        config["epochs"] = 2
        print(f"⚠️ DEBUG mode enabled — running only {config['epochs']} epochs\n")


    print("\n🚀🚀🚀 Starting training process... 🚀🚀🚀\n")
    df_train = df_stage2_train
    train_with_groupkfold(
        df_train             = df_train,
        save_dir             = model_root,
        config               = config,
        device               = device
    )
    print(f"\n✅ Training completed! Results saved in: {model_root}")
    print("✅"*65)

    model_dir = DIRS["save"]
    out_dir = DIRS["model"]
    model_paths = sorted(Path(model_dir).glob("*.pt"))
    print(f": Found {len(model_paths)} unique models!")
    for path in model_paths:
        print(f" - {path}")
    if len(model_paths) < 2:
        print(f"⚠️ Only {len(model_paths)} unique models, skipping SWA...")
    
    models = []
    for module_path in model_paths:
        if os.path.exists(module_path):
            model = torch.load(module_path, map_location='cpu')
            models.append(model)
        else:
            print(f"❌ Model file not found: {module_path}")
    
    worker_state_dicts = [m for m in models]
    weight_keys = list(worker_state_dicts[0].keys())
    print(f"Example weight keys: {list(weight_keys)[:5]}")
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        key_sum = 0
        for i in range(len(models)):
            key_sum += worker_state_dicts[i][key]
        fed_state_dict[key] = key_sum / len(models)
    os.makedirs(out_dir, exist_ok=True)
    output_path = os.path.join(out_dir, f'swa.pt')
    torch.save(fed_state_dict, output_path)
    print(f"averaging complete. Saved to: {output_path}")



def load_and_prepare_test_df():
    df_file_path = Path(DIRS["dir"]) / "test.csv"
    df = pd.read_csv(df_file_path)

    df["ID"] = df["sample_id"].str.split("__").str[0]
    df = move_column_first(df, "ID")
    df["target"] = 0  
    df_targets = (
        df.pivot_table(index="ID", columns="target_name", values="target", aggfunc="first")
        .reset_index()
    )
    df_targets.columns.name = None
    df_meta = df[["ID", "image_path"]].drop_duplicates(subset="ID")
    df_test = pd.merge(df_meta, df_targets, on="ID", how="left")
    show_df_info(df_test, "df_test")
    print(f"✅ Test set loaded: {df_test.shape}")
    return df_test


def load_model_for_inference(model_path, device, config):
    model = MyDualStreamModel(config["backbone_name"], pretrained=False, config=config)
    model = model.to(device).to(memory_format=torch.channels_last)
    try:
        state_dict = torch.load(model_path, map_location=device, weights_only=True)
    except TypeError:
        state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model


def predict_with_model(model, dataloader, device):
    preds_list = []
    model.eval()
    with torch.no_grad():
        for step, (img_left, img_right, targets) in enumerate(dataloader):
            img_left   = img_left.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last)
            img_right  = img_right.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last)
            targets    = targets.to(device, non_blocking=True)
            with torch.amp.autocast(device_type='cuda'):
                preds = model(img_left, img_right)
            preds_list.append(preds.cpu().numpy())
    return np.concatenate(preds_list, axis=0)


def predict_ensemble(df_test, pre_transform, transform, model_dir, device, config):
    model_paths = sorted(Path(model_dir).glob("*.pt"))
    assert len(model_paths) > 0, f"❌ No model files found in: {model_dir}"
    print(f"✅ Detected {len(model_paths)} 个模型:")
    for p in model_paths:
        print("   -", p.name)
    test_dataset = DualStreamDataset(
        df_test,
        DIRS["dir"],
        config,
        pre_transform=pre_transform,
        transform=transform,
        mode="test"
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )
    fold_preds = []
    for fold, model_path in enumerate(model_paths):
        print(f"Fold {fold+1}/{len(model_paths)} 推理: {model_path.name}")
        model = load_model_for_inference(model_path, device, config)
        fold_pred = predict_with_model(model, test_loader, device)
        fold_preds.append(fold_pred)
    preds_mean = np.mean(fold_preds, axis=0)
    df_pred = pd.DataFrame(preds_mean, columns=config["target_cols"])
    df_pred["ID"] = df_test["ID"]
    df_pred = df_pred[["ID"] + config["target_cols"]]
    show_df_info(df_pred, "df_pred")
    return df_pred


def generate_submission(df_pred_final):
    ordered_target_cols = [
        "Dry_Green_g",  
        "Dry_Clover_g",
        "Dry_Dead_g",
        "GDM_g",       
        "Dry_Total_g"
    ]
    # if "Dry_Clover_g" in df_pred_final.columns:
    #     df_pred_final["Dry_Clover_g"] = df_pred_final["Dry_Clover_g"] * 0.8
    # else:
    #     print("   ⚠️ Warning: Column Dry_Clover_g not found in DataFrame")

    df_submit = (
        df_pred_final.melt(
            id_vars="ID",
            value_vars=ordered_target_cols,
            var_name="target_name",
            value_name="target"
        )
    )

    df_submit["sample_id"] = df_submit["ID"] + "__" + df_submit["target_name"]
    df_submit = df_submit[["sample_id", "target"]]
    df_submit = df_submit.sort_values("sample_id").reset_index(drop=True)
    df_submit.to_csv("submission_stage2_2.csv", index=False)
    print("\n✅ Submission file generated: submission_stage2_2.csv")


def run_tta_prediction(df_test, model_dir, device, config):
    tta_transforms = get_tta_transforms(config["img_size"])
    tta_names = list(tta_transforms.keys())
    print(f"\n✅ Detected {len(tta_names)} TTA modes: {tta_names}\n")
    all_preds = []
    for name, _ in tta_transforms.items():
        print(f"\n🚀 TTA mode: {name}")
        if name == "base":
            print("base no pre_transform")
            pre_transform = None
            transform = tta_transforms[name]
        else:
            pre_transform = tta_transforms[name]
            transform = tta_transforms["base"]
        df_pred = predict_ensemble(df_test, pre_transform, transform, model_dir, device, config)
        all_preds.append(df_pred[config["target_cols"]].values)

    mean_preds = np.mean(all_preds, axis=0)
    df_pred_final = df_pred.copy()
    df_pred_final[config["target_cols"]] = mean_preds
    return df_pred_final


if __name__ == "__main__" and isPREDICT:
    print("\n🧠 Starting prediction pipeline...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    df_test = load_and_prepare_test_df()
    print(f"\nCurrent dataset shape: {df_test.shape}\n")
    model_dir = DIRS["model"]

    print(f"Model directory loaded: {model_dir}")

    df_pred_final = run_tta_prediction(df_test, model_dir, device, config)
    generate_submission(df_pred_final)
    print("🎯 Prediction pipeline completed.")

Writing train_dual.py


In [11]:
%%bash
CUDA_VISIBLE_DEVICES=0 python train_single.py > log_single_train.txt 2>&1 &
CUDA_VISIBLE_DEVICES=1 python train_dual.py > log_dual_train.txt 2>&1 &
echo "GPU 0: Single Stream"
echo "GPU 1: Dual Stream"

wait

echo "single&&dual done！"

GPU 0: Single Stream
GPU 1: Dual Stream
single&&dual done！


In [12]:
!tail -n 1000 log_single_train.txt

/usr/local/lib/python3.11/dist-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context

In [13]:
!tail -n 1000 log_dual_train.txt

/usr/local/lib/python3.11/dist-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context

In [14]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('submission_stage2_1.csv')
df2 = pd.read_csv('submission_stage2_2.csv')

df1 = df1.sort_values('sample_id').reset_index(drop=True)
df2 = df2.sort_values('sample_id').reset_index(drop=True)

if not df1['sample_id'].equals(df2['sample_id']):
    print("警告：两个文件的sample_id不完全一致！")
    print("将按sample_id对齐...")
    df_merged = pd.merge(df1, df2, on='sample_id', suffixes=('_1', '_2'))
else:
    df_merged = df1.copy()
    df_merged['target_1'] = df1['target']
    df_merged['target_2'] = df2['target']


df_merged['target'] = 0.6 * df_merged['target_1'] + 0.4 * df_merged['target_2']
df_merged['target'] = df_merged['target'].apply(lambda x: max(x, 0) if x >= 0.1 else 0)
df_merged['target'] = df_merged['target'].clip(lower=0)
result_df = df_merged[['sample_id', 'target']].copy()
result_df.to_csv('submission_stage2.csv', index=False)
print(f"\n✅ 提交文件已保存: submission_stage2.csv")
print(result_df.head(10))


✅ 提交文件已保存: submission_stage2.csv
                    sample_id     target
0  ID1001187975__Dry_Clover_g   0.123179
1    ID1001187975__Dry_Dead_g  32.610937
2   ID1001187975__Dry_Green_g  37.190625
3   ID1001187975__Dry_Total_g  69.735866
4         ID1001187975__GDM_g  37.476492


In [15]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('submission_stage1.csv')
df2 = pd.read_csv('submission_stage2.csv')

df1 = df1.sort_values('sample_id').reset_index(drop=True)
df2 = df2.sort_values('sample_id').reset_index(drop=True)

if not df1['sample_id'].equals(df2['sample_id']):
    print("警告：两个文件的sample_id不完全一致！")
    print("将按sample_id对齐...")
    df_merged = pd.merge(df1, df2, on='sample_id', suffixes=('_1', '_2'))
else:
    df_merged = df1.copy()
    df_merged['target_1'] = df1['target']
    df_merged['target_2'] = df2['target']


df_merged['target'] = 0.2 * df_merged['target_1'] + 0.8 * df_merged['target_2']
df_merged['target'] = df_merged['target'].apply(lambda x: max(x, 0) if x >= 0.2 else 0)
df_merged['target'] = df_merged['target'].clip(lower=0)
result_df = df_merged[['sample_id', 'target']].copy()

negative_count = (result_df['target'] < 0).sum()
below_01_count = ((df_merged['target_1'] < 0.2) | (df_merged['target_2'] < 0.2)).sum()
clipped_count = (result_df['target'] == 0).sum()

print(f"处理后统计:")
print(f"  负值数量: {negative_count}")
print(f"  原始值低于0.1的数量: {below_01_count}")
print(f"  被置为0的数量: {clipped_count}")
print(f"  最终值范围: [{result_df['target'].min():.3f}, {result_df['target'].max():.3f}]")

result_df.to_csv('submission.csv', index=False)
print(f"\n✅ 提交文件已保存: submission.csv")

print(result_df.head(10))

处理后统计:
  负值数量: 0
  原始值低于0.1的数量: 1
  被置为0的数量: 1
  最终值范围: [0.000, 68.902]

✅ 提交文件已保存: submission.csv
                    sample_id     target
0  ID1001187975__Dry_Clover_g   0.000000
1    ID1001187975__Dry_Dead_g  32.024312
2   ID1001187975__Dry_Green_g  36.914500
3   ID1001187975__Dry_Total_g  68.902161
4         ID1001187975__GDM_g  37.122600
